## Installing demoji

In [1]:
!pip install demoji

## Imports

In [2]:
import pandas as pd
import numpy as np

import keras

from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Loading Datasets

In [3]:
df_train = pd.read_excel('.....',names=["ID","Tweets","Labels"])
df_train.dropna(inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_val = pd.read_csv(".....",names=["ID","Tweets","Labels"])
df_val.dropna(inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [4]:
df_train_Tweet = pd.DataFrame(df_train.Tweets)
df_val_Tweet = pd.DataFrame(df_val.Tweets)
df_train_Label = pd.DataFrame(df_train.Labels)
df_val_Label = pd.DataFrame(df_val.Labels)


## Preprocessing

In [5]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
import demoji

def preprocessing(document):
        document = demoji.replace_with_desc(document).replace(":"," ").replace("-"," ")
        document = regex.sub(' ', document)
        document = re.sub(r'[0-9]', '', document)
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)             # remove all single characters
        document = re.sub(r'\s+', ' ', document, flags=re.I)            # Substituting multiple spaces with single space
        return document

In [6]:
corpus_train = df_train_Tweet.Tweets.apply(preprocessing)
corpus_val = df_val_Tweet.Tweets.apply(preprocessing)

## TF-IDF Vectorization

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,5))
cv = TfidfVectorizer(max_features=15000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_val).toarray()
X_train.shape

(3999, 15000)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Label Encoding

In [9]:
Y_train = np.ravel(df_train_Label)
Y_test = np.ravel(df_val_Label)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
Y_test = le.transform(Y_test)

# Model Creation and Validation

## SIMPLE ANN

In [10]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(512, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               7680512   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 7,844,865
Trainable params: 7,844,865
Non-trainable params: 0
_________________________________________________________________


In [12]:
model_save_path='/content/m1'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [13]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=30,
                    callbacks=callbacks)

Epoch 1/35
134/134 [==============================] - 13s 8ms/step - loss: 0.6921 - accuracy: 0.5515 - val_loss: 0.6893 - val_accuracy: 0.6702
Epoch 2/35
134/134 [==============================] - 1s 6ms/step - loss: 0.6758 - accuracy: 0.9000 - val_loss: 0.6666 - val_accuracy: 0.7713
Epoch 3/35
134/134 [==============================] - 1s 5ms/step - loss: 0.6048 - accuracy: 0.9432 - val_loss: 0.5593 - val_accuracy: 0.8032
Epoch 4/35
134/134 [==============================] - 1s 6ms/step - loss: 0.3847 - accuracy: 0.9575 - val_loss: 0.4206 - val_accuracy: 0.8223
Epoch 5/35
134/134 [==============================] - 1s 5ms/step - loss: 0.1692 - accuracy: 0.9799 - val_loss: 0.3718 - val_accuracy: 0.8245
Epoch 6/35
134/134 [==============================] - 1s 6ms/step - loss: 0.0810 - accuracy: 0.9909 - val_loss: 0.3607 - val_accuracy: 0.8319
Epoch 7/35
134/134 [==============================] - 1s 5ms/step - loss: 0.0503 - accuracy: 0.9913 - val_loss: 0.3619 - val_accuracy: 0.8330
Epoch

In [14]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.83      0.83      0.83       465
         OFF       0.84      0.83      0.83       475

    accuracy                           0.83       940
   macro avg       0.83      0.83      0.83       940
weighted avg       0.83      0.83      0.83       940



###Creating padded sequences

In [15]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(corpus_train)

X_train = tokenizer.texts_to_sequences(corpus_train)
X_test = tokenizer.texts_to_sequences(corpus_val)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(corpus_train[2])
print(X_train[2])

 kalimuthu ne ena lusa yaaru edhu panaalum en da religion ah ilukreenga manishana paaru da modhalla 
[7014, 25, 38, 2235, 249, 1162, 7015, 27, 5, 1163, 8, 7016, 7017, 141, 5, 1042]


In [16]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 120

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## ANN with Embedding matrix

In [17]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [18]:
optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 100)          1668500   
_________________________________________________________________
flatten (Flatten)            (None, 12000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               6144512   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 7,977,365
Trainable params: 7,977,365
Non-trainable params: 0
____________________________________________

In [19]:
model_save_path='/content/m2'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [20]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 4s 22ms/step - loss: 0.6869 - accuracy: 0.5469 - val_loss: 0.5835 - val_accuracy: 0.6862
Epoch 2/30
125/125 [==============================] - 2s 19ms/step - loss: 0.3294 - accuracy: 0.8656 - val_loss: 0.4455 - val_accuracy: 0.7936
Epoch 3/30
125/125 [==============================] - 2s 19ms/step - loss: 0.0840 - accuracy: 0.9708 - val_loss: 0.4179 - val_accuracy: 0.8426
Epoch 4/30
125/125 [==============================] - 2s 19ms/step - loss: 0.0216 - accuracy: 0.9920 - val_loss: 0.4471 - val_accuracy: 0.8277
Epoch 5/30
125/125 [==============================] - 2s 18ms/step - loss: 0.0052 - accuracy: 0.9923 - val_loss: 0.4135 - val_accuracy: 0.8521
Epoch 6/30
125/125 [==============================] - 2s 20ms/step - loss: 0.0028 - accuracy: 0.9935 - val_loss: 0.5056 - val_accuracy: 0.8191
Epoch 7/30
125/125 [==============================] - 2s 18ms/step - loss: 0.0044 - accuracy: 0.9931 - val_loss: 0.6211 - val_accuracy: 0.8074

In [21]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.85      0.85      0.85       465
         OFF       0.85      0.86      0.85       475

    accuracy                           0.85       940
   macro avg       0.85      0.85      0.85       940
weighted avg       0.85      0.85      0.85       940



####Reduced dimension to 50

In [22]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 50)           834250    
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 834,771
Trainable params: 834,771
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_save_path='/content/m3'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [24]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 2s 11ms/step - loss: 0.6847 - accuracy: 0.6272 - val_loss: 0.6227 - val_accuracy: 0.6745
Epoch 2/30
125/125 [==============================] - 1s 9ms/step - loss: 0.5429 - accuracy: 0.8806 - val_loss: 0.4176 - val_accuracy: 0.8415
Epoch 3/30
125/125 [==============================] - 1s 9ms/step - loss: 0.2667 - accuracy: 0.9401 - val_loss: 0.3331 - val_accuracy: 0.8553
Epoch 4/30
125/125 [==============================] - 1s 9ms/step - loss: 0.1285 - accuracy: 0.9654 - val_loss: 0.3048 - val_accuracy: 0.8691
Epoch 5/30
125/125 [==============================] - 1s 9ms/step - loss: 0.0595 - accuracy: 0.9863 - val_loss: 0.3147 - val_accuracy: 0.8617
Epoch 6/30
125/125 [==============================] - 1s 9ms/step - loss: 0.0341 - accuracy: 0.9887 - val_loss: 0.3245 - val_accuracy: 0.8617
Epoch 7/30
125/125 [==============================] - 1s 9ms/step - loss: 0.0143 - accuracy: 0.9923 - val_loss: 0.3295 - val_accuracy: 0.8638
Epoch

In [25]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.91      0.82      0.86       465
         OFF       0.84      0.92      0.88       475

    accuracy                           0.87       940
   macro avg       0.87      0.87      0.87       940
weighted avg       0.87      0.87      0.87       940



##CNN

In [26]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 100)          1668500   
_________________________________________________________________
conv1d (Conv1D)              (None, 120, 64)           19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [27]:
model_save_path='/content/m4'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [28]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 32s 20ms/step - loss: 0.6921 - accuracy: 0.5203 - val_loss: 0.6832 - val_accuracy: 0.6266
Epoch 2/30
125/125 [==============================] - 2s 16ms/step - loss: 0.6061 - accuracy: 0.7674 - val_loss: 0.3852 - val_accuracy: 0.8415
Epoch 3/30
125/125 [==============================] - 2s 17ms/step - loss: 0.2539 - accuracy: 0.9057 - val_loss: 0.2793 - val_accuracy: 0.8798
Epoch 4/30
125/125 [==============================] - 2s 17ms/step - loss: 0.1081 - accuracy: 0.9639 - val_loss: 0.2834 - val_accuracy: 0.8745
Epoch 5/30
125/125 [==============================] - 2s 16ms/step - loss: 0.0517 - accuracy: 0.9875 - val_loss: 0.3364 - val_accuracy: 0.8543
Epoch 6/30
125/125 [==============================] - 2s 16ms/step - loss: 0.0285 - accuracy: 0.9922 - val_loss: 0.4017 - val_accuracy: 0.8383
Epoch 7/30
125/125 [==============================] - 2s 17ms/step - loss: -0.0141 - accuracy: 0.9881 - val_loss: 0.3756 - val_accuracy: 0.85

In [29]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.87      0.89      0.88       465
         OFF       0.89      0.87      0.88       475

    accuracy                           0.88       940
   macro avg       0.88      0.88      0.88       940
weighted avg       0.88      0.88      0.88       940



In [30]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 50)           834250    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 120, 64)           9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [31]:
model_save_path='/content/m5'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [32]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 3s 15ms/step - loss: 0.6919 - accuracy: 0.5256 - val_loss: 0.6729 - val_accuracy: 0.7489
Epoch 2/30
125/125 [==============================] - 1s 11ms/step - loss: 0.6032 - accuracy: 0.7581 - val_loss: 0.3313 - val_accuracy: 0.8947
Epoch 3/30
125/125 [==============================] - 1s 11ms/step - loss: 0.2654 - accuracy: 0.9025 - val_loss: 0.2731 - val_accuracy: 0.8798
Epoch 4/30
125/125 [==============================] - 1s 12ms/step - loss: 0.1397 - accuracy: 0.9578 - val_loss: 0.2750 - val_accuracy: 0.8787
Epoch 5/30
125/125 [==============================] - 1s 12ms/step - loss: 0.0768 - accuracy: 0.9767 - val_loss: 0.2776 - val_accuracy: 0.8787
Epoch 6/30
125/125 [==============================] - 1s 11ms/step - loss: 0.0439 - accuracy: 0.9819 - val_loss: 0.3026 - val_accuracy: 0.8755
Epoch 7/30
125/125 [==============================] - 1s 11ms/step - loss: 0.0209 - accuracy: 0.9883 - val_loss: 0.3231 - val_accuracy: 0.8745

In [33]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.85      0.95      0.90       465
         OFF       0.95      0.84      0.89       475

    accuracy                           0.89       940
   macro avg       0.90      0.90      0.89       940
weighted avg       0.90      0.89      0.89       940



## BILSTM

In [34]:
embedding_dim = 100
max_len = 120

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 100)          1668500   
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 512)          731136    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [42]:
model_save_path='/content/m6'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [43]:
history = model.fit(X_train, Y_train,
                     epochs=50,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks = callbacks)

Epoch 1/50
125/125 [==============================] - 4s 34ms/step - loss: 0.5107 - accuracy: 0.8147 - val_loss: 0.7179 - val_accuracy: 0.4957
Epoch 2/50
125/125 [==============================] - 4s 34ms/step - loss: 0.4735 - accuracy: 0.8710 - val_loss: 0.7942 - val_accuracy: 0.5543
Epoch 3/50
125/125 [==============================] - 4s 35ms/step - loss: 0.4643 - accuracy: 0.8955 - val_loss: 0.5583 - val_accuracy: 0.7745
Epoch 4/50
125/125 [==============================] - 5s 36ms/step - loss: 0.4313 - accuracy: 0.9337 - val_loss: 0.8471 - val_accuracy: 0.5085
Epoch 5/50
125/125 [==============================] - 4s 35ms/step - loss: 0.4189 - accuracy: 0.9482 - val_loss: 0.7700 - val_accuracy: 0.5787
Epoch 6/50
125/125 [==============================] - 4s 35ms/step - loss: 0.4051 - accuracy: 0.9640 - val_loss: 0.7149 - val_accuracy: 0.6234
Epoch 7/50
125/125 [==============================] - 4s 35ms/step - loss: 0.3966 - accuracy: 0.9757 - val_loss: 0.6803 - val_accuracy: 0.6596

In [44]:
model.load_weights(model_save_path)

class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.81      0.78      0.79       465
         OFF       0.79      0.82      0.80       475

    accuracy                           0.80       940
   macro avg       0.80      0.80      0.80       940
weighted avg       0.80      0.80      0.80       940



####Reduced dimension to 50

In [45]:
embedding_dim = 50
max_len = 120

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 50)           834250    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 512)          628736    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_5 (Dropout)          (None, 20)               

In [46]:
model_save_path='/content/m7'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [47]:
history = model.fit(X_train, Y_train,
                     epochs=100,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/100
125/125 [==============================] - 8s 34ms/step - loss: 0.6956 - accuracy: 0.4893 - val_loss: 0.6677 - val_accuracy: 0.5053
Epoch 2/100
125/125 [==============================] - 3s 26ms/step - loss: 0.6299 - accuracy: 0.6670 - val_loss: 0.6927 - val_accuracy: 0.4989
Epoch 3/100
125/125 [==============================] - 3s 26ms/step - loss: 0.5928 - accuracy: 0.8824 - val_loss: 0.6268 - val_accuracy: 0.8287
Epoch 4/100
125/125 [==============================] - 3s 27ms/step - loss: 0.5642 - accuracy: 0.9199 - val_loss: 0.5951 - val_accuracy: 0.8117
Epoch 5/100
125/125 [==============================] - 3s 26ms/step - loss: 0.5422 - accuracy: 0.9440 - val_loss: 0.5861 - val_accuracy: 0.8032
Epoch 6/100
125/125 [==============================] - 3s 25ms/step - loss: 0.5215 - accuracy: 0.9601 - val_loss: 0.5844 - val_accuracy: 0.7904
Epoch 7/100
125/125 [==============================] - 3s 25ms/step - loss: 0.5041 - accuracy: 0.9619 - val_loss: 0.6935 - val_accuracy:

In [48]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.79      0.93      0.86       465
         OFF       0.92      0.76      0.83       475

    accuracy                           0.84       940
   macro avg       0.85      0.85      0.84       940
weighted avg       0.86      0.84      0.84       940

